In [1]:
import logging
import os
import openai
import itertools


from actionweaver.llms.openai.tools.chat import OpenAIChatCompletion
from actionweaver.llms.openai.tools.tokens import TokenUsageTracker
from actionweaver import action

openai.api_key = os.getenv("OPENAI_API_KEY")


 

def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')


In [2]:
logging.basicConfig(
    filename='parallel_tools.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [3]:

@action(name="GetWeather")
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


In [4]:
chat = OpenAIChatCompletion("gpt-3.5-turbo-1106", logger=logger)

In [6]:
print_output(chat.create([{"role": "user", "content": "what time is it now and What's the weather like in San Francisco, Tokyo, and Paris?"}], actions = [], stream=False))

> /Users/tenghu/Code/actiontools/actionweaver/llms/openai/tools/chat.py(284)create()
    282             pdb.set_trace()
    283 
--> 284             if message.tool_calls:
    285                 tools, (stop, resp) = self._invoke_tool(
    286                     call_id,



ipdb>  pp api_response


ChatCompletion(id='chatcmpl-8WCdHNVdrYkU8cir7xYcji02Lenuw', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="I'm sorry, I cannot provide real-time information such as the current time and weather. I recommend checking a reliable weather website or app for the most up-to-date information.", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1702685747, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=35, prompt_tokens=27, total_tokens=62))


ipdb>  api_response.model_dump()


{'id': 'chatcmpl-8WCdHNVdrYkU8cir7xYcji02Lenuw', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': "I'm sorry, I cannot provide real-time information such as the current time and weather. I recommend checking a reliable weather website or app for the most up-to-date information.", 'role': 'assistant', 'function_call': None, 'tool_calls': None}, 'logprobs': None}], 'created': 1702685747, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_772e8125bb', 'usage': {'completion_tokens': 35, 'prompt_tokens': 27, 'total_tokens': 62}}


ipdb>  q


In [6]:
print_output(get_current_time.invoke(chat, [{"role": "user", "content": "what time is it ?"}], stream=True))


Getting current time
The current time is 16:53:24.

In [9]:
from pydantic import BaseModel
from actionweaver.actions.factories.pydantic_model_to_action import action_from_model

class User(BaseModel):
    name: str
    age: int

action_from_model(User, stop=True).invoke(chat, [{"role": "user", "content": "Tom is 31 years old"}])

[User(name='Tom', age=31)]

In [6]:
print_output(chat.create([{"role": "user", "content": "what time is it now"}], actions = [get_current_time], orch = {get_current_time: get_current_weather}, stream=False))






Getting current time


OpenAIChatCompletionException: GetWeather is not a valid tool name, use one of the following: GetWeather